`William Untung`

In [1]:
pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [3]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/1GF9dqm5fxAJb9KOV6tdFoUx8cM1f1-eukI4ce-ytadI/export?format=csv&gid=0")

In [4]:
df.head()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,Cameron Williams,42,11066.80,0,7.22,8,2013-08-30 7:00:40,"10265 Elizabeth Mission Barkerburgh, AK 89518",Harvey LLC,1
1,Kevin Mueller,41,11916.22,0,6.50,11,2013-08-13 0:38:46,"6157 Frank Gardens Suite 019 Carloshaven, RI 1...",Wilson PLC,1
2,Eric Lozano,38,12884.75,0,6.67,12,2016-06-29 6:20:07,"1331 Keith Court Alyssahaven, DE 90114","Miller, Johnson and Wallace",1
3,Phillip White,42,8010.76,0,6.71,10,2014-04-22 12:43:12,"13120 Daniel Mount Angelabury, WY 30645-4695",Smith Inc,1
4,Cynthia Norton,37,9191.58,0,5.56,9,2016-01-19 15:31:15,"765 Tricia Row Karenshire, MH 71730",Love-Jones,1


In [30]:
print(f'Dataset Shape: {df.shape}\n')
print(f'Null Values :\n{df.isna().sum()}')

Dataset Shape: (900, 10)

Null Values :
Names              0
Age                0
Total_Purchase     0
Account_Manager    0
Years              0
Num_Sites          0
Onboard_date       0
Location           0
Company            0
Churn              0
dtype: int64


In [32]:
print(f"Duplicated Values : {df.duplicated().sum()}")

Duplicated Values : 0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Names            900 non-null    object 
 1   Age              900 non-null    int64  
 2   Total_Purchase   900 non-null    float64
 3   Account_Manager  900 non-null    int64  
 4   Years            900 non-null    float64
 5   Num_Sites        900 non-null    int64  
 6   Onboard_date     900 non-null    object 
 7   Location         900 non-null    object 
 8   Company          900 non-null    object 
 9   Churn            900 non-null    int64  
dtypes: float64(2), int64(4), object(4)
memory usage: 70.4+ KB


In [8]:
df.describe()

,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
count,900.000000,900.000000,900.000000,900.000000,900.000000,900.000000
mean,41.816667,10062.824033,0.481111,5.273156,8.587778,0.166667
std,6.127560,2408.644532,0.499921,1.274449,1.764836,0.372885
min,22.000000,100.000000,0.000000,1.000000,3.000000,0.000000
25%,38.000000,8497.122500,0.000000,4.450000,7.000000,0.000000
50%,42.000000,10045.870000,0.000000,5.215000,8.000000,0.000000
75%,46.000000,11760.105000,1.000000,6.110000,10.000000,0.000000
max,65.000000,18026.010000,1.000000,9.150000,14.000000,1.000000


In [9]:
for i in df.columns:
  print(f'{df[i].value_counts()}\n')

Names
Jennifer Wood        2
Mary Marshall        1
Thomas Campbell      1
Christopher Clark    1
Rose White           1
                    ..
Sandra Mcintyre      1
Robert Joseph        1
George Chavez        1
Joshua Walters       1
Rodney Evans         1
Name: count, Length: 899, dtype: int64

Age
41    69
43    59
40    58
45    56
44    53
38    51
42    49
37    48
46    48
39    48
36    39
48    36
35    32
49    30
47    29
34    25
51    21
52    20
32    18
33    16
50    15
55    14
30    11
31    11
29     9
53     8
54     8
56     5
28     5
58     2
26     2
65     1
60     1
25     1
27     1
22     1
Name: count, dtype: int64

Total_Purchase
9315.60     1
11066.80    1
11916.22    1
12884.75    1
6406.38     1
           ..
14062.60    1
9885.12     1
11331.58    1
10356.02    1
9191.58     1
Name: count, Length: 900, dtype: int64

Account_Manager
0    467
1    433
Name: count, dtype: int64

Years
5.89    8
5.07    8
6.45    7
5.51    7
4.61    7
       ..
2.64    1


Analysis:
- Names : Drop, <br>This feature is an identifier and does not contain predictive information. Keeping it may introduce noise and negatively affect model performance.
- Age : Use, <br>A numerical feature that may influence customer behavior and churn tendency.
- Total_Purchase : Use, <br>A numerical feature that represents customer value and is expected to have an impact on churn prediction.
- Account_Manager : Use, <br>A boolean feature indicating whether a customer is assigned an account manager. This feature may influence customer retention.
- Num_Sites : Use, <br>A numerical feature representing the number of sites used by the customer. A higher number of sites may indicate stronger dependency on the service.
- Onboard_date : Drop, <br>Although potentially useful, this feature is not used because the data collection date is unknown. Without a clear reference time, it is not possible to derive a reliable tenure feature.
- Location : Drop, <br>This feature is considered irrelevant for churn prediction, as location varies across customers and does not show a clear relationship with churn behavior.
- Company : Drop, <br>This feature has high cardinality, where most company values are unique and only a small number of companies appear multiple times (1 or more occurrences). As a result, it does not provide meaningful patterns for the model and may introduce noise.
- Churn : Label, <br>The target variable indicating whether a customer has churned (1) or not (0).

In [10]:
df_final = df.drop(columns = ['Names', 'Onboard_date', 'Location', 'Company'])

In [11]:
df_final['Churn'].value_counts()

,count
Churn,
0,750
1,150


Dataset is imbalanced

In [12]:
x = df_final.drop(columns = ['Churn'], axis = 1)
y = df_final['Churn']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [13]:
num_cols = ['Age', 'Total_Purchase', 'Years', 'Num_Sites']
bin_cols = ['Account_Manager']

preprocess_logistic = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('bin', 'passthrough', bin_cols)
    ]
)

lr = LogisticRegression(
    max_iter = 1000,
    class_weight='balanced'
)

model_logistic = Pipeline(steps=[
    ('preprocess', preprocess_logistic),
    ('clf', lr)
])

In [14]:
model_logistic.fit(x_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'Total_Purchase',
                                                   'Years', 'Num_Sites']),
                                                 ('bin', 'passthrough',
                                                  ['Account_Manager'])])),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [15]:
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['liblinear', 'lbfgs'],
    'clf__class_weight': ['balanced', None]
}

In [16]:
tune_model_logistic1 = GridSearchCV(
    estimator = model_logistic,
    param_grid = param_grid,
    cv = 5,
    scoring = 'recall',
    n_jobs = -1
)

tune_model_logistic2 = GridSearchCV(
    estimator = model_logistic,
    param_grid = param_grid,
    cv = 5,
    scoring = 'roc_auc',
    n_jobs = -1
)

In [17]:
tune_model_logistic1.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'Total_Purchase',
                                                                          'Years',
                                                                          'Num_Sites']),
                                                                        ('bin',
                                                                         'passthrough',
                                                                         ['Account_Manager'])])),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.01, 0.1, 1, 10],
                         'clf__class_weight': ['balanced', None],
                         'clf__penalty': ['l1', 'l2'],
                         'clf__solver': ['liblinear', 'lbfgs']},
             scoring='recall')

In [18]:
tune_model_logistic2.fit(x_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
40 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **l

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'Total_Purchase',
                                                                          'Years',
                                                                          'Num_Sites']),
                                                                        ('bin',
                                                                         'passthrough',
                                                                         ['Account_Manager'])])),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.01, 0.1, 1, 10],
                         'clf__class_weight': ['balanced', None],
                         'clf__penalty': ['l1', 'l2'],
                         'clf__solver': ['liblinear', 'lbfgs']},
             scoring='roc_auc')

In [19]:
y_pred_logistic = model_logistic.predict(x_test)

In [20]:
print(f'Accuracy Score (Base Model) : {accuracy_score(y_test, y_pred_logistic)}')
print("Precision (Base Model) : ", precision_score(y_test, y_pred_logistic))
print("Recall (Base Model) : ", recall_score(y_test, y_pred_logistic))
print("F1_score (Base Model) : ", f1_score(y_test, y_pred_logistic))

Accuracy Score (Base Model) : 0.8388888888888889
Precision (Base Model) :  0.5306122448979592
Recall (Base Model) :  0.8125
F1_score (Base Model) :  0.6419753086419753


In [21]:
y_proba_logistic = model_logistic.predict_proba(x_test)[:, 1]
print("ROC-AUC (Base Model) : ", roc_auc_score(y_test, y_proba_logistic))

ROC-AUC (Base Model) :  0.8992820945945945


In [22]:
y_pred_logistic_tune1 = tune_model_logistic1.predict(x_test)

In [23]:
print(f'Accuracy Score (Tune Model) : {accuracy_score(y_test, y_pred_logistic_tune1)}')
print("Precision (Tune Model) : ", precision_score(y_test, y_pred_logistic_tune1))
print("Recall (Tune Model) : ", recall_score(y_test, y_pred_logistic_tune1))
print("F1_score (Tune Model) : ", f1_score(y_test, y_pred_logistic_tune1))

Accuracy Score (Tune Model) : 0.6722222222222223
Precision (Tune Model) :  0.3411764705882353
Recall (Tune Model) :  0.90625
F1_score (Tune Model) :  0.49572649572649574


In [24]:
y_proba_logistic_tune1 = tune_model_logistic1.predict_proba(x_test)[:, 1]
print("ROC-AUC (Tune Model) : ", roc_auc_score(y_test, y_proba_logistic_tune1))

ROC-AUC (Tune Model) :  0.8532516891891891


In [25]:
y_pred_logistic_tune2 = tune_model_logistic2.predict(x_test)

In [26]:
print(f'Accuracy Score (Tune Model) : {accuracy_score(y_test, y_pred_logistic_tune2)}')
print("Precision (Tune Model) : ", precision_score(y_test, y_pred_logistic_tune2))
print("Recall (Tune Model) : ", recall_score(y_test, y_pred_logistic_tune2))
print("F1_score (Tune Model) : ", f1_score(y_test, y_pred_logistic_tune2))

Accuracy Score (Tune Model) : 0.8277777777777777
Precision (Tune Model) :  1.0
Recall (Tune Model) :  0.03125
F1_score (Tune Model) :  0.06060606060606061


In [27]:
y_proba_logistic_tune2 = tune_model_logistic2.predict_proba(x_test)[:, 1]
print("ROC-AUC (Tune Model) : ", roc_auc_score(y_test, y_proba_logistic_tune2))

ROC-AUC (Tune Model) :  0.8967483108108109


In [28]:
result = pd.DataFrame({
    'Model' : ['Base Model', 'Tune Model1', 'Tune Model2'],
    'Accuracy' : [0.8388888888888889, 0.6722222222222223, 0.8277777777777777],
    'Precision' : [0.5306122448979592, 0.3411764705882353, 1.0],
    'Recall' : [0.8125, 0.90625, 0.03125],
    'F1_Score' : [0.6419753086419753, 0.49572649572649574, 0.06060606060606061],
    'AUC' : [0.8992820945945945, 0.8967483108108109, 0.8967483108108109]
})

In [29]:
result

,Model,Accuracy,Precision,Recall,F1_Score,AUC
0,Base Model,0.838889,0.530612,0.81250,0.641975,0.899282
1,Tune Model1,0.672222,0.341176,0.90625,0.495726,0.896748
2,Tune Model2,0.827778,1.000000,0.03125,0.060606,0.896748
